In [1]:
import pandas as pd
import numpy as np
import joblib

In [2]:
mofid = pd.read_csv("../data/main_dataset_mofid_data_V2.csv", index_col=0)
mofid = mofid[["smiles_linkers", "smiles_nodes", "cifname"]].set_index("cifname")
mordred = pd.read_csv("../data/all_f_main_dataset_mordred_V2.csv", index_col=0)
zeo = pd.read_csv("../data/main_dataset_zeo_V2.csv", index_col=0)
cif = pd.read_csv("../data/main_dataset_cif_property_V2.csv", index_col=0)



In [3]:
mordred.shape, zeo.shape, cif.shape

((151, 1826), (162, 3), (165, 8))

In [4]:
def make_mordred():
    from mordred import Calculator, descriptors
    from rdkit import Chem
    mofid = pd.read_csv("../data/main_dataset_mofid_data_V2.csv", index_col=0)
    mofid = mofid[["smiles_linkers", "smiles_nodes", "cifname"]].set_index("cifname")
    linkers = [mofid.smiles_linkers[i].replace("'", "")[1:-1].split(", ") for i in range(mofid.__len__())]
    mols = [[Chem.MolFromSmiles(smi) for smi in smi_list] for smi_list in linkers]
    calc = Calculator(descriptors, ignore_3D=False)
    def f(mof):
        try: return calc.pandas(mof)
        except TypeError:
            return None
    dfs = [f(mof) for mof in mols]
    data_mordred = pd.DataFrame(columns=dfs[0].columns)

    for i, filename in enumerate(mofid.index):
        try:
            if linkers[i] != [""]:
                data_mordred.loc[filename] = dfs[i].mean()
        except AttributeError:
            print(f"{filename:_^20}")
    data_mordred = data_mordred.set_index(data_mordred.index.map(lambda name: name.replace(" ", "")))
    #data_mordred.to_csv("../data/all_f_main_dataset_mordred_V2.csv")
    return data_mordred



In [5]:
def metal_from_node(node: str):
    import re
    """
    input: smilesNodes: str
    return: 
    'metals' in node: list
    'unique' types of metals: np.array, dtype='<U2'
    'count' of unique: np.array, dtype=int
    """
    # "O[Zr]123(O)[OH]4[Zr]56([O]3[Zr]37([OH]2[Zr]28([O]1[Zr]14([O]6[Zr]([OH]53)([OH]21)([O]78)(O)O)([OH2])([OH2])O)[OH2])([OH2])([OH2])O)[OH2]"
    node = node.replace("OH", "").replace("O", "")
    node = node.replace("[", "").replace("]", "").replace(")", "").replace("(", "").replace(",", "")
    node = re.sub(r"\d", "", node) # replace numbers
    #print(node)
    # "ZrZrZrZrZrZr"
    start_cut = 0
    metals = []
    for i, char in enumerate(node[1:]):
        if not char.islower():
            metals.append(node[start_cut:i+1])
            start_cut = i+1
    metals.append(node[start_cut:])
    unique, counts = np.unique(np.array(metals), return_counts=True)
    return metals, unique, counts

In [6]:
elemental_descriptors = pd.read_csv("../data/elemental_descriptors.csv")

In [7]:
node_descriptors = pd.DataFrame(columns=("n_metals", 'n_types_metals', 'Atomic_Number', 'Atomic_Weight', 'Atomic Radius',
       'Mulliken EN', 'polarizability(A^3)', 'electron affinity(kJ/mol)'))

d = ['Atomic_Number', 'Atomic_Weight', 'Atomic Radius',
       'Mulliken EN', 'polarizability(A^3)', 'electron affinity(kJ/mol)']

for filename, node in zip(mofid.index.map(lambda name: name.replace(" ", "").replace(".cif", "")), mofid["smiles_nodes"]):
    try:
       metals, unique, count = metal_from_node(node.replace("'", "").replace(" ", "").replace("%", ""))
       #print(metals)
       n_metals = count.sum()
       n_metals_types = count.__len__()
       df = pd.DataFrame(columns=d, index=[range(count.sum())])
       for metal in metals:
          #print(metal)
          df.loc[len(df)] = elemental_descriptors.loc[elemental_descriptors["Symbol"] == metal].loc[:,d].iloc[0]
       node_descriptors.loc[filename] = n_metals, n_metals_types, *df.mean().array
    except IndexError:
       print(f"Error with {node}")

Error with []
Error with []
Error with []
Error with []
Error with ['*']
Error with ['*']
Error with []
Error with []
Error with ['*']
Error with ['*']
Error with ['*']
Error with ['*']
Error with ['*']
Error with ['*']
Error with ['*']


In [8]:
node_descriptors = node_descriptors.loc[node_descriptors["n_types_metals"] == 1.]


In [9]:
zeo.index = zeo.index.map(lambda s: s.replace(".res", ""))

In [10]:
data_pld_lcd = zeo[["num1", "num2"]].rename({"num1": "lcd", "num2": "pld"}, axis=1)

In [11]:
cif = cif.rename({ "sg_number": "spacegroupNumber"}, axis=1)

In [12]:
cif.index = cif.index.map(lambda name: name.replace(" ", "").replace(".cif", ""))

In [13]:
cif = cif.set_index(cif.index.map(lambda name: name.replace("(", "").replace(")", "")))

In [14]:
node_descriptors = node_descriptors.set_index(node_descriptors.index.map(lambda name: name.replace("(", "").replace(")", "")))

In [15]:
data_pld_lcd = data_pld_lcd.set_index(data_pld_lcd.index.map(lambda name: name.replace("(", "").replace(")", "")))

In [16]:
mordred = mordred.set_index(mordred.index.map(lambda name: name.replace("(", "").replace(")", "")))

In [17]:
data_pld_lcd.shape, cif.shape, mordred.shape, node_descriptors.shape

((162, 2), (165, 8), (151, 1826), (145, 8))

In [18]:
data_main = pd.concat([data_pld_lcd, cif, mordred, node_descriptors], axis=1)

In [19]:
data_main = data_main.loc[list({*list(data_pld_lcd.index)} & {*list(
    cif.index)} & {*list(mordred.index)} & {*list(node_descriptors.index)})]


In [20]:
data_main.shape

(138, 1844)

In [21]:
# data_main.to_csv("../data/main_no_marked.csv")

In [22]:
db_from_to = pd.read_csv("../data/DB_main_with_SG_for_EDA.csv", sep=';')

In [23]:
db_from_to.CIF_init = db_from_to.CIF_init.map(lambda name: name.replace("(", "").replace(")", "").replace(" ", "").replace(".cif", ""))
db_from_to.CIF_final = db_from_to.CIF_final.map(lambda name: name.replace("(", "").replace(")", "").replace(" ", "").replace(".cif", ""))


In [24]:
marked = pd.DataFrame(columns=["target"])

for ind in db_from_to.index:
    if db_from_to.loc[ind, "Stimuli"].find("solvent") != -1:
        marked.loc[db_from_to.loc[ind, "CIF_init"]] = [1]
        if db_from_to.loc[ind, "Reversible"] == "yes":
            marked.loc[db_from_to.loc[ind, "CIF_final"]] = [1]
        else:
            marked.loc[db_from_to.loc[ind, "CIF_final"]] = [0]
    else:
        marked.loc[db_from_to.loc[ind, "CIF_init"]] = [0]
        marked.loc[db_from_to.loc[ind, "CIF_final"]] = [0]

In [25]:
from pteproc_model import PreprocessingModel
model = joblib.load("../qmof_datasets/scaler.pkl")

In [26]:
data_main.shape

(138, 1844)

In [27]:
main_dataset = data_main[model.cols]

In [28]:
marked_dataset = pd.concat([data_main, marked], axis=1)

In [29]:
marked_dataset = marked_dataset.loc[list({*list(main_dataset.index)} & {*list(marked.index)})]

In [30]:
marked_dataset.shape

(102, 1845)

In [31]:
target, counts = np.unique(marked_dataset.target, return_counts=True)


print(f"|{'target':^10}|{'counts':^10}|")
print(f"{'-':-^23}")
print(f"|{'yes':^10}|{counts[0]:^10}|")
print(f"|{'no':^10}|{counts[1]:^10}|")
print(f"total count:{marked_dataset.__len__()}")

|  target  |  counts  |
-----------------------
|   yes    |    65    |
|    no    |    37    |
total count:102


In [32]:
marked_dataset.shape

(102, 1845)

In [33]:
x, y = model.transform(marked_dataset.drop(['target'], axis=1)), marked_dataset['target']

In [34]:
x.shape, x.dropna().shape

((102, 1158), (102, 1158))

In [35]:
x

pld       lcd         a         b  \
from31483716                     0.085791  0.071084  0.071471  0.478036   
fromMOF2fa1052168                0.145899  0.136053  0.290554  0.259259   
toCCDC_2065040_JUK-20-nor        0.224280  0.227874  0.377709  0.377076   
fromFebdp25C1058446              0.065594  0.047184  0.106673  0.067158   
to3D854461                       0.176924  0.181942  0.117070  0.693169   
...                                   ...       ...       ...       ...   
fromassynthesizedDUT-30Zn804770  0.179165  0.170028  0.189917  0.285933   
from1withDMF1403842              0.109974  0.273373  0.473247  0.442781   
to5cm6b05277_si_004              0.072625  0.061039  0.238585  0.375420   
toDMFsolvatedFebdp298K1058448    0.308066  0.270376  0.304245  0.268205   
toDUT-49Cd100K2014972            0.359271  0.620280  1.295803  1.292465   

                                        c     alpha      beta     gamma  \
from31483716                     0.287759  0.578143  0.526960  0.441638   
fromMOF2fa1052168                0.138619  0.524217  0.201038  0.356215   
toCCDC_2065040_JUK-20-nor        0.218935  0.631468  0.526960  0.441638   
fromFebdp25C1058446              0.007664  0.595239  0.750304  0.441638   
to3D854461                       0.434415  0.578143  0.526960  0.441638   
...                                   ...       ...       ...       ...   
fromassynthesizedDUT-30Zn804770  0.156803  0.510829  0.258064  0.186620   
from1withDMF1403842              0.263726  0.341554  0.206129  0.062258   
to5cm6b05277_si_004              0.217806  0.899960  0.526960  0.441638   
toDMFsolvatedFebdp298K1058448    0.144718  0.578143  0.526960  0.559041   
toDUT-49Cd100K2014972            0.842953  0.578143  0.526960  0.441638   

                                   volume  spacegroupNumber  ...   Zagreb2  \
from31483716                     0.045042          0.233480  ...  0.094290   
fromMOF2fa1052168                0.040242          0.004405  ...  0.124834   
toCCDC_2065040_JUK-20-nor        0.107066          0.052863  ...  0.075033   
fromFebdp25C1058446              0.008816          0.061674  ...  0.025232   
to3D854461                       0.120914          0.074890  ...  0.529880   
...                                   ...               ...  ...       ...   
fromassynthesizedDUT-30Zn804770  0.033658          0.299559  ...  0.282869   
from1withDMF1403842              0.141524          0.678414  ...  0.151394   
to5cm6b05277_si_004              0.052769          0.845815  ...  0.095618   
toDMFsolvatedFebdp298K1058448    0.047179          0.083700  ...  0.128818   
toDUT-49Cd100K2014972            1.998402          0.898678  ...  0.467463   

                                 mZagreb1  mZagreb2  n_metals  Atomic_Number  \
from31483716                     0.093381  0.105263  0.000000       0.292135   
fromMOF2fa1052168                0.132758  0.112782  0.090909       0.269663   
toCCDC_2065040_JUK-20-nor        0.061504  0.068922  0.000000       0.505618   
fromFebdp25C1058446              0.000000  0.011278  0.000000       0.258427   
to3D854461                       0.459966  0.551378  0.090909       0.303371   
...                                   ...       ...       ...            ...   
fromassynthesizedDUT-30Zn804770  0.218264  0.268170  0.090909       0.303371   
from1withDMF1403842              0.176261  0.155388  0.818182       0.303371   
to5cm6b05277_si_004              0.103131  0.092732  0.000000       0.280899   
toDMFsolvatedFebdp298K1058448    0.059254  0.115288  0.000000       0.258427   
toDUT-49Cd100K2014972            0.383274  0.431078  0.090909       0.505618   

                                 Atomic_Weight  Atomic Radius  Mulliken EN  \
from31483716                          0.244950       0.150000     0.852234   
fromMOF2fa1052168                     0.224989       0.188889     0.790378   
toCCDC_2065040_JUK-20-nor             0.456406       0.238889     0.800687   
fromFebdp25C1058446       

In [36]:
x.to_csv("../main_datasets/dataset.csv")

In [37]:
y.to_csv("../main_datasets/target.csv")